---

[//]: # "Header"

<p style="text-align: left;  font-size:18pt; LINE-HEIGHT:30px;">
    <span style="float: left">
     Technische Hochschule Ingolstadt<br>
     Prof. Dr. Sören Gröttrup<br>
     Laura Dietl
    </span>
    <span style="float: right;">
       Machine Learning 1<br>
        <span style="float: right;">WS 24/25</span>
    </span>
</p>

[//]: # "Header Aufgabenblatt"
<br>
<p style="text-align: center;  font-size:18pt; LINE-HEIGHT:30px;">
     <span style="font-weight: bold;">Aufgabenblatt 5</span><br>
     Themen: ROC<br>
     Abgabetermin: 13.12.2024, 23:59 Uhr <br>
    Punkte: 20
</p>


---

**Namen:** Emelie Hauck, Conrad Christoph

---

# 1. ROC

In [33]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, roc_auc_score

## Aufgabe 1.1 [10 Punkte]
1. Schreiben Sie eine Funktion `calc_roc(y_labels, pred_prob)`, welche die Werte der ROC-Kurve (FPR, TPR , threshold) für ein binäre Klassifikation berechnet und diese zurückgibt. Die Eingaben `y_labels` enthält dabei die wahren Werte eine Zielvariablen und `pred_pob` ist ein Verktor mit Wahrscheinlichkeiten für die Zugehörigkeit des jeweiligen Sample zur positiven Klassen. Beispiele sind unterhalb der Aufgabe angegeben.
1. Erweitern Sie die Funktion so, dass sie zusätzlich den AUC berechnet und diesen ebenfalls zurückgibt, sprich `roc_values, auc = calc_roc(y_labels, pred_prob)`
1. Plotten Sie die ROC-Kurve für die unten agegebenen Vektoren `y_labels` und `pred_prob` und berechnen Sie die AUC
1. Bei welchen Schwellwert (Threshold) für die Entscheidung ob ein Sample zur Klasse 1 gehört ist die Accuracy am größten?

_Hinweis:_ 
* Für die Aufgabe dürfen Sie nicht die von Python (sklearn) bereitgestellten Funktionen für die Berechnung des roc und auc verwenden, sondern sollen diese selbständig entwickeln.
* Python stellt die Funktion `sklearn.metrics.roc_curve` zur Berechnung der TPR und FPR zur Verfügung. Sie können diese Funktion als Kontrolle nutzen, um die Richtigkeit Ihrer Berechnungen zu verifizieren.

In [34]:
y_labels = np.load(file="Daten Blatt 5/target.npy")
pred_prob = np.load(file="Daten Blatt 5/pred.npy")

In [197]:
#### Code HERE ####

def get_single_tpr_fpr(df):

    '''
    Note, this implementation is only for binaly class labels (0 and 1)
    :param df: the dataframe should have 'y' and 'y_predicted' as its labels
    :return: a list containing tpr and fpr
    '''

    tp = ((df['y'] == 1.0 ) & (df['y_predicted'] == 1)).sum()
    fp = ((df['y'] == 0.0 ) & (df['y_predicted'] == 1)).sum()
    tn = ((df['y'] == 0.0 ) & (df['y_predicted'] == 0)).sum()
    fn = ((df['y'] == 1.0 ) & (df['y_predicted'] == 0)).sum()

    tpr = tp / (tp + fn )
    fpr = fp / (fp + tn)

    return [tpr, fpr]


# While computing AUC score you need to calculate "TP,"FP" at every threshold by using actual "y" and predicted "y_pred".

def calculate_all_thresholds_tpr_fpr_arr(df_original):

    '''

    :param df_original: the original dataframe, which should have a 'proba' label
    :return: two arrays, tpr_arr_for_all_thresholds, fpr_arr_for_all_thresholds
    '''

    tpr_arr_for_all_thresholds = []
    fpr_arr_for_all_thresholds = []

    sorted_df = df_original.sort_values(by=['proba'], ascending=False)

    unique_probability_thresholds = sorted_df['proba'].unique()

    for threshold in tqdm(unique_probability_thresholds):
        sorted_df['y_predicted'] = np.where(sorted_df['proba'] >= threshold, 1, 0)
        tpr_fpr_arr = get_single_tpr_fpr(sorted_df)
        tpr_arr_for_all_thresholds.append(tpr_fpr_arr[0])
        fpr_arr_for_all_thresholds.append(tpr_fpr_arr[1])

    return tpr_arr_for_all_thresholds, fpr_arr_for_all_thresholds







def cal_roc(y_labels, pred_prob):
    
    thresholds = np.unique(pred_prob)[::-1]
    print(thresholds)
    tprs = []
    fprs = []
    for theta in thresholds:
        pred = (pred_prob >= theta).astype(int)
        
        TP = np.sum((pred == 1) & (y_labels == 1))
        FP = np.sum((pred == 1) & (y_labels == 0))
        TN = np.sum((pred == 0) & (y_labels == 0))
        FN = np.sum((pred == 0) & (y_labels == 1))
    
        TPR = TP / (TP + FN) #if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) #if (FP + TN) > 0 else 0

        tprs.append(TPR)
        fprs.append(FPR)
    
    control_fprs, control_tprs, control_thresholds = roc_curve(y_labels, pred_prob)

    tprs = np.array(tprs)
    print(tprs)
    fprs = np.array(fprs)
    thresholds = np.array(thresholds)

    tpr_diff = np.diff(tprs)
    fpr_diff = np.diff(fprs)

    auc = np.sum(tprs[:-1] * fpr_diff) + 0.5 * np.sum(fpr_diff * (tpr_diff + tprs[:-1]))
    auc = np.trapz(fprs[::-1], tprs[::-1])
    auc_control = roc_auc_score(y_labels, pred_prob)
    print(auc_control)

    return (tprs, fprs, thresholds), auc


roc_val, auc = cal_roc(y_labels, pred_prob)
print(auc)

[9.97774845e-01 9.95735795e-01 9.93515061e-01 9.93358040e-01
 9.92338916e-01 9.91180568e-01 9.90418743e-01 9.90341936e-01
 9.90145578e-01 9.89323108e-01 9.87427675e-01 9.87346618e-01
 9.87018367e-01 9.86417092e-01 9.86178377e-01 9.86004312e-01
 9.85891181e-01 9.85862706e-01 9.85415984e-01 9.85224815e-01
 9.84506017e-01 9.84376649e-01 9.84224417e-01 9.83915331e-01
 9.83729366e-01 9.83693560e-01 9.83139249e-01 9.82085725e-01
 9.80832341e-01 9.79313109e-01 9.79233315e-01 9.79168920e-01
 9.78645790e-01 9.78564089e-01 9.77927616e-01 9.76764826e-01
 9.76339259e-01 9.75890746e-01 9.75770238e-01 9.75587375e-01
 9.74699016e-01 9.74655534e-01 9.74150887e-01 9.72992370e-01
 9.72067357e-01 9.72053444e-01 9.71765709e-01 9.70845918e-01
 9.70606677e-01 9.69287732e-01 9.69255765e-01 9.69010297e-01
 9.67570941e-01 9.67553122e-01 9.67148703e-01 9.66603209e-01
 9.66362429e-01 9.65982597e-01 9.65777175e-01 9.64575766e-01
 9.64145289e-01 9.64038410e-01 9.63456563e-01 9.62875675e-01
 9.62131626e-01 9.619789

## Aufgabe 1.2 [10 Punkte]

**Daten:** Der Datensatz `creditcard_small.csv` ist ein gekürzter Datensatz mit weniger Variablen von dem Original-Datensatz https://www.kaggle.com/mlg-ulb/creditcardfraud. Er enthält anonymisierte Informationen von Kreditkartentransaktionen und ein Label `Class`, ob diese Transaktion betrügerisch war (`Class=1`), oder nicht (`Class=2`).

1. Trainieren Sie das Logistisches-Regrassionsmodell `log_reg = LogisticRegression()` (siehe Code unten) auf den Trainingsdaten `x_train, y_train` und machen Sie damit eine Vorhersage für die Testdaten `x_test`.
1. Berechnen Sie diverse Evaluationsmetriken für de Performance des Modells auf den Testdaten mittels einen Klassifikations-Report durch die Funktion `classification_report` aus dem Modul `sklearn.metrics`. Was ist die Accuracy, Precision, Sensitivity, Specificity und F1-Score für das Modell?
1. Berechnen Sie die AUC und plotten Sie die ROC-Kurve für die Testdaten. Interpretieren Sie dessen Verlauf. Sie können hier die Funktionen `sklearn.metrics.roc_curve` und `sklearn.metrics.roc_auc_score` benutzen.
1. Trainieren Sie das Neuronale Netz `nn = MLPClassifier(activation="relu")` auf den Trainigsdaten `x_train, y_train`. Wie ist die AUC bei diesem Modell auf den Testdaten? Plotten Sie auch hier die ROC-Kurve und vergleichen Sie diese mit der der Logistischen Regression.

**Vorgeschriebener Code. Bitte nicht verändern**

In [198]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.style.use('ggplot')

In [199]:
#load data
data = pd.read_csv("Daten Blatt 5/creditcard_small.csv")
data

,V5,V6,V7,V8,V27,V28,Class
0,-0.338321,0.462388,0.239599,0.098698,0.133558,-0.021053,0
1,0.060018,-0.082361,-0.078803,0.085102,-0.008983,0.014724,0
2,-0.503198,1.800499,0.791461,0.247676,-0.055353,-0.059752,0
3,-0.010309,1.247203,0.237609,0.377436,0.062723,0.061458,0
4,-0.407193,0.095921,0.592941,-0.270533,0.219422,0.215153,0
...,...,...,...,...,...,...,...
284802,-5.364473,-2.606837,-4.918215,7.305334,0.943651,0.823731,0
284803,0.868229,1.058415,0.024330,0.294869,0.068472,-0.053527,0
284804,2.630515,3.031260,-0.296827,0.708417,0.004455,-0.026561,0
284805,-0.377961,0.623708,-0.686180,0.679145,0.108821,0.104533,0


In [200]:
#Split into train and test
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size=0.3, random_state=1)

In [201]:
y_train = data_train.Class
x_train = data_train.drop(columns=["Class"])

y_test = data_test.Class
x_test = data_test.drop(columns=["Class"])

In [202]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
log_reg = LogisticRegression()
nn = MLPClassifier(activation="relu")

**Ihren Code ab hier einfügen**

In [203]:
#### CODE HERE ####
...